<a href="https://colab.research.google.com/github/mikful/dlai-tf-in-practice/blob/master/TensorFlow_in_Practice_Week_1_Exercise4_Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
# Import libraries

import tensorflow as tf
import os
import zipfile

In [2]:
#Download dataset and unzip

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

data_dir = "/tmp/h-or-s" # desired data directory

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall(data_dir)
zip_ref.close()

--2020-01-21 23:15:41--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.01s   

2020-01-21 23:15:42 (265 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [0]:
#Create callback

DESIRED_ACCURACY = 0.999

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc')>DESIRED_ACCURACY):
      print(f'\n Training stopped as accuracy exceeded {DESIRED_ACCURACY*100} %.')
      self.model.stop_training = True

callbacks = myCallback()

In [0]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')   
])


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc']
              )

In [14]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255) # normalize images

train_generator = train_datagen.flow_from_directory(
    data_dir,  # The image source directory to flow files from
    batch_size = 16, # batch_size
    target_size=(150, 150),  # resize images to 150x150px x 3RGB channels
    class_mode='binary' # binary_crossentropy loss as using binary labels
    )

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [15]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit_generator(
    train_generator, # Training data from generator
    epochs = 10, # number of epochs to train
    callbacks = [callbacks]
    )
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/10
5/5 [==============================] - 1s 132ms/step - loss: 0.8268 - acc: 0.5500
Epoch 2/10
5/5 [==============================] - 0s 46ms/step - loss: 0.6390 - acc: 0.6125
Epoch 3/10
5/5 [==============================] - 0s 41ms/step - loss: 0.3459 - acc: 0.9375
Epoch 4/10
5/5 [==============================] - 0s 38ms/step - loss: 0.2824 - acc: 0.8875
Epoch 5/10
5/5 [==============================] - 0s 39ms/step - loss: 0.1247 - acc: 0.9500
Epoch 6/10
5/5 [==============================] - 0s 40ms/step - loss: 0.0899 - acc: 0.9625
Epoch 7/10
5/5 [==============================] - 0s 40ms/step - loss: 0.0817 - acc: 0.9625
Epoch 8/10
5/5 [==============================] - 0s 42ms/step - loss: 0.0885 - acc: 0.9750
Epoch 9/10
5/5 [==============================] - 0s 35ms/step - loss: 0.0667 - acc: 0.9625
Epoch 10/10
3/5 [=================>............] - ETA: 0s - loss: 0.0212 - acc: 1.0000
 Training stopped as accuracy exceeded 99.9 %.
5/5 [==============================]